In [1]:
import csv
import cv2
import numpy as np

lines = []
with open('./data/data/driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    for line in reader:
        lines.append(line)
#         break



In [2]:
from sklearn.model_selection import train_test_split
train_valid, test = train_test_split(lines, test_size=0.01)
# train_samples, validation_samples = train_test_split(train_valid, test_size=0.1)


In [3]:
images = []
measurments = []
correction = [0.0,0.2,-0.2]
import random
print(random.randint(0,9))
dropped_samples = 0
for line in train_valid:
    for idx in range(3):
#         if float(line[3]) == 0.0 and (idx == 0 and random.randint(0,9) < -1):
#             dropped_samples=dropped_samples+1
#         else:
        source_path = line[idx]
        filename = source_path.split('/')[-1]
        current_path = './data/data/IMG/' + filename
        image = cv2.imread(current_path)
        vimage=image.copy()
        vimage=cv2.flip(image,1)
        images.append(image)
        images.append(vimage)
        measurment = float(line[3]) + correction[idx]
        measurments.append(measurment)
        measurments.append(-measurment)
    
print("dropped samples: " +str(dropped_samples))

4
dropped samples: 0


In [4]:
X_train = np.array(images)
y_train = np.array(measurments)
print(X_train.shape)

(47730, 160, 320, 3)


In [5]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, Dropout


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
model.add(Conv2D(8,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(16,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(32,(5,5),activation="relu"))
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(120,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
n_epoch = 10
n_batch = 128
for i in range(n_epoch):
    model.fit(X_train, y_train, batch_size=n_batch, validation_split=0.05, shuffle=True)
model.save('model.h5')
# exit()



Using TensorFlow backend.


Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 109s 2ms/step - loss: 0.0187 - val_loss: 0.0113
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 103s 2ms/step - loss: 0.0143 - val_loss: 0.0104
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 103s 2ms/step - loss: 0.0129 - val_loss: 0.0099
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 104s 2ms/step - loss: 0.0117 - val_loss: 0.0092
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 103s 2ms/step - loss: 0.0107 - val_loss: 0.0090
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [==============================] - 105s 2ms/step - loss: 0.0098 - val_loss: 0.0081
Train on 45343 samples, validate on 2387 samples
Epoch 1/1
45343/45343 [====

In [6]:
images = []
measurments = []
correction = [0.0,0.2,-0.2]
import random
print(random.randint(0,9))
dropped_samples = 0
for line in test:
    for idx in range(3):
#         if float(line[3]) == 0.0 and (idx == 0 and random.randint(0,9) < -1):
#             dropped_samples=dropped_samples+1
#         else:
        source_path = line[idx]
        filename = source_path.split('/')[-1]
        current_path = './data/data/IMG/' + filename
        image = cv2.imread(current_path)
        vimage=image.copy()
        vimage=cv2.flip(image,1)
        images.append(image)
        images.append(vimage)
        measurment = float(line[3]) + correction[idx]
        measurments.append(measurment)
        measurments.append(-measurment)
        
X_test = np.array(images)
y_test = np.array(measurments)

9


In [7]:
import tensorflow as tf
with tf.device('/cpu:0'):
    from keras.models import load_model
    model = load_model('model.h5')
    score = model.evaluate(x=X_test, y=y_test, batch_size=32)
    print('Test loss:', score)
    

486/486 [==============================] - 6s 13ms/step
Test loss: 0.005827484998292455
